In [39]:
from bertopic import BERTopic
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from opensearch_data_model import Topic, TopicKeyword, os_client
from datetime import datetime
from dateutil.parser import parse
from utils import SPANISH_STOPWORDS
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd
import joblib

In [40]:
articles_to_predict = pd.read_parquet('train-00000-of-00001.parquet')

In [41]:
articles_to_predict

,asset_id,title_ch,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,entities_transformers,title,text,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay
0,112042673,"Antes de reunirse con el Papa, Kicillof ratifi...",Infobae,2691693,2024-06-18 12:55:26,2024-06-18 09:55:26,"[Axel Kicillof, Frente Renovador, La Cámpora, ...","[Victoria Villarruel, Aysa, Federico Achával, ...",2024-06-18 13:01:05.044436,"[{'entities': [{'end': 29, 'entity_group': 'PE...","[Papa, Kicillof, Congreso, Facundo Cottet Nuev...","Antes de reunirse con el Papa, Kicillof ratifi...","18 Jun, 2024 Por Facundo Cottet Nuevo El gobe...","[plaza, inmediaciones, ley bases, rechazo, vot...",2024-06-18 13:06:02.245420,"18 Jun, 2024 Por Facundo Cottet Nuevo El gobe...","Antes de reunirse con el Papa, Kicillof ratifi...",0.082556,0.176735
1,112024515,Maestros de Colombia inician huelga en protest...,Infobae,394888,2024-06-18 06:24:31,2024-06-18 03:24:31,[Gustavo Petro],"[Gustavo Petro, Fecode, Aurora Vergara, Petro,...",2024-06-18 06:25:21.130514,"[{'entities': [{'end': 20, 'entity_group': 'LO...","[Colombia, Newsroom Infobae Nuevo, Bogotá, EFE...",Maestros de Colombia inician huelga en protest...,"18 Jun, 2024 Por Newsroom Infobae Nuevo Bogot...","[educación pública, distintos gobiernos, huelg...",2024-06-18 06:27:09.502660,"18 Jun, 2024 Por Newsroom Infobae Nuevo Bogot...",Maestros de Colombia inician huelga en protest...,0.030103,0.044029
2,112047372,Rabanne une fuerzas con Art Basel y presenta `...,Infobae,1665815,2024-06-18 13:58:07,2024-06-18 10:58:07,[Casa],"[Julien Dossena, Art Basel, CBO, Club, Central...",2024-06-18 15:13:25.450971,"[{'entities': [{'end': 7, 'entity_group': 'ORG...","[Rabanne, Art Basel, The Rabanne Arts Factory,...",Rabanne une fuerzas con Art Basel y presenta '...,"18 Jun, 2024 Por Newsroom Infobae Nuevo Duran...","[arte contemporáneo, imágenes, feria art, venu...",2024-06-18 15:13:50.359720,"18 Jun, 2024 Por Newsroom Infobae Nuevo Duran...",Rabanne une fuerzas con Art Basel y presenta '...,0.006919,1.262044
3,112072772,Insistencia de pesca industrial en la Reserva ...,Infobae,119808,2024-06-18 18:14:07,2024-06-18 15:14:07,[],"[Ruiz, Lucía Ruiz, Sernanp, Sociedad Nacional ...",2024-06-18 18:14:24.113955,"[{'entities': [{'end': 56, 'entity_group': 'LO...","[Reserva de Paracas, Ambiente, Daniela Valdivi...",Insistencia de pesca industrial en la Reserva ...,"18 Jun, 2024 Por Daniela Valdivia Blume Nuevo...","[prohibición, pesca, sala constitucional, inte...",2024-06-18 18:22:43.008770,"18 Jun, 2024 Por Daniela Valdivia Blume Nuevo...",Insistencia de pesca industrial en la Reserva ...,0.138582,0.143336
4,112013930,Jin de BTS terminó servicio militar surcoreano...,Infobae,295527,2024-06-18 03:50:35,2024-06-18 00:50:35,[],"[Jungkook, J-Hope, Namjoon, Suga, Jin, Pilar C...",2024-06-18 03:50:57.301956,"[{'entities': [{'end': 3, 'entity_group': 'PER...","[Jin, BTS, Pilar Cuya, Kim Seok-Jin, Corea del...",Jin de BTS terminó servicio militar surcoreano...,"18 Jun, 2024 Por Pilar Cuya Nuevo Kim Seok-Ji...","[reencuentro, actividades individuales, servic...",2024-06-18 03:57:12.071060,"18 Jun, 2024 Por Pilar Cuya Nuevo Kim Seok-Ji...",Jin de BTS terminó servicio militar surcoreano...,0.104103,0.110298
5,112015419,Jugada ganadora y resultado del último sorteo ...,Infobae,1277955,2024-06-18 04:16:06,2024-06-18 01:16:06,[],"[Azar, Chao Chamba, La Tinka, Ministerio]",2024-06-18 04:16:47.381847,"[{'entities': [{'end': 55, 'entity_group': 'OR...","[Kábala, Infobae Noticias Nuevo, La Tinka, Cha...",Jugada ganadora y resultado del último sorteo ...,"18 Jun, 2024 Por Infobae Noticias Nuevo Como ...","[sorteo, apuestas deportivas, jugada ganadora,...",2024-06-18 04:20:25.161820,"18 Jun, 2024 Por Infobae Noticias Nuevo Como ...",Jugada ganadora y resultado del último sorteo ...,0.060494,0.071989
6,112013973,Super Once: estos son los resul

In [42]:
topic_model = joblib.load('topic_model')

In [43]:
topic_pred,prob = topic_model.transform(articles_to_predict['title_and_text'])

In [44]:
articles_to_predict['prediction'] = topic_pred
articles_to_predict['prob'] = prob
articles_to_predict['prediction'] = articles_to_predict.apply(lambda x: -1 if x['prob']<0.2 else x['prediction'],axis=1)

In [45]:
articles_to_predict['keywords_matched'] = ''
articles_to_predict['entities_matched'] = ''
for i in np.arange(len(articles_to_predict)):
    matched_keywords=[]
    matched_entities=[]
    if articles_to_predict.loc[i,'prediction']>-1:
        query = {
            "query": {
                "match": {
                    "index": articles_to_predict.loc[i,'prediction']
                }
            }
        }
        response = os_client.search(index='topic', body=query)
        for j in articles_to_predict.loc[i,'keywords']:
            if j in response['hits']['hits'][0]['_source']['keywords']:
                matched_keywords.append(j)
        articles_to_predict['keywords_matched'][i] = matched_keywords

        for j in articles_to_predict.loc[i,'entities']:
            if j in response['hits']['hits'][0]['_source']['entities']:
                matched_entities.append(j)
        articles_to_predict['entities_matched'][i] = matched_entities


c:\Users\gaspa\anaconda3\envs\topics\Lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\gaspa\anaconda3\envs\topics\Lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\gaspa\anaconda3\envs\topics\Lib\site-packages\urllib3\connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\gaspa\anaconda3\envs\topics\Lib

In [46]:
articles_to_predict

,asset_id,title_ch,media,impact,start_time_utc,start_time_local,entities_curated,entities,predicted_at_entities,entities_raw_transformers,...,keywords,predicted_at_keywords,truncated_text,title_and_text,prediction_delay_predictions,prediction_delay,prediction,prob,keywords_matched,entities_matched
0,112042673,"Antes de reunirse con el Papa, Kicillof ratifi...",Infobae,2691693,2024-06-18 12:55:26,2024-06-18 09:55:26,"[Axel Kicillof, Frente Renovador, La Cámpora, ...","[Victoria Villarruel, Aysa, Federico Achával, ...",2024-06-18 13:01:05.044436,"[{'entities': [{'end': 29, 'entity_group': 'PE...",...,"[plaza, inmediaciones, ley bases, rechazo, vot...",2024-06-18 13:06:02.245420,"18 Jun, 2024 Por Facundo Cottet Nuevo El gobe...","Antes de reunirse con el Papa, Kicillof ratifi...",0.082556,0.176735,30,0.318197,"[ley bases, rechazo, fuerzas, mandatario, proy...",[Axel Kicillof]
1,112024515,Maestros de Colombia inician huelga en protest...,Infobae,394888,2024-06-18 06:24:31,2024-06-18 03:24:31,[Gustavo Petro],"[Gustavo Petro, Fecode, Aurora Vergara, Petro,...",2024-06-18 06:25:21.130514,"[{'entities': [{'end': 20, 'entity_group': 'LO...",...,"[educación pública, distintos gobiernos, huelg...",2024-06-18 06:27:09.502660,"18 Jun, 2024 Por Newsroom Infobae Nuevo Bogot...",Maestros de Colombia inician huelga en protest...,0.030103,0.044029,37,0.400050,"[presidente, trabajadores, sindicato, proyecto...",[Gustavo Petro]
2,112047372,Rabanne une fuerzas con Art Basel y presenta `...,Infobae,1665815,2024-06-18 13:58:07,2024-06-18 10:58:07,[Casa],"[Julien Dossena, Art Basel, CBO, Club, Central...",2024-06-18 15:13:25.450971,"[{'entities': [{'end': 7, 'entity_group': 'ORG...",...,"[arte contemporáneo, imágenes, feria art, venu...",2024-06-18 15:13:50.359720,"18 Jun, 2024 Por Newsroom Infobae Nuevo Duran...",Rabanne une fuerzas con Art Basel y presenta '...,0.006919,1.262044,-1,0.000000,,
3,112072772,Insistencia de pesca industrial en la Reserva ...,Infobae,119808,2024-06-18 18:14:07,2024-06-18 15:14:07,[],"[Ruiz, Lucía Ruiz, Sernanp, Sociedad Nacional ...",2024-06-18 18:14:24.113955,"[{'entities': [{'end': 56, 'entity_group': 'LO...",...,"[prohibición, pesca, sala constitucional, inte...",2024-06-18 18:22:43.008770,"18 Jun, 2024 Por Daniela Valdivia Blume Nuevo...",Insistencia de pesca industrial en la Reserva ...,0.138582,0.143336,33,0.682932,"[argumentos, ambiente, canal, sector, núcleo, ...",[]
4,112013930,Jin de BTS terminó servicio militar surcoreano...,Infobae,295527,2024-06-18 03:50:35,2024-06-18 00:50:35,[],"[Jungkook, J-Hope, Namjoon, Suga, Jin, Pilar C...",2024-06-18 03:50:57.301956,"[{'entities': [{'end': 3, 'entity_group': 'PER...",...,"[reencuentro, actividades individuales, servic...",2024-06-18 03:57:12.071060,"18 Jun, 2024 Por Pilar Cuya Nuevo Kim Seok-Ji...",Jin de BTS terminó servicio militar surcoreano...,0.104103,0.110298,-1,0.000000,,
5,112015419,Jugada ganadora y resultado del último sorteo ...,Infobae,1277955,2024-06-18 04:16:06,2024-06-18 01:16:06,[],"[Azar, Chao Chamba, La Tinka, Ministerio]",2024-06-18 04:16:47.381847,"[{'entities': [{'end': 55, 'entity_group': 'OR...",...,"[sorteo, apuestas deportivas, jugada ganadora,...",2024-06-18 04:20:25.161820,"18 Jun, 2024 Por Infobae Noticias Nuevo Como ...",Jugada ganadora y resultado del último sorteo ...,0.060494,0.071989,70,0.918660,"[sorteo, resultados, premios, aciertos, premio...",[]
6,112013973,Super Once: estos son los resultados ganadores...,Infobae,296645,2024-06-18 03:51:58,2024-06-18 00:51:58,[],"[Ju, ONCE]",2024-06-18 03:52:13.695400,"[{'entities': [{'end': 10, 'entity_group': 'MI...",...,"[combinación ganadora, super, reciente sorteo,...",2024-06-18 03:57:33.961670,"18 Jun, 2024 Por Rodrigo Gutiérrez González N...",Super Once: estos son los resultados ganadores...,0.088963,0.093323,70,1.000000,"[combinación ganadora, euros, premios, premio,...",[]
7,112043263,En el comienzo de la negociación del nuevo pro...,Infobae,1617891,2024-06-18 13:08:21,2024-06-18 10:08:

In [ ]:
articles_to_predict.to_parquet('train-00000-of-00001 with predictions.parquet')